In [40]:
# 文件夹中有很多照片/视频等文件
# 希望能够按照“修改日期”进行归档（创建以日期为名字的目录，并将该日期修改的文件移动至其中）
# 之所以不选择“创建日期”，是因为创建日期在文件移动（如复制到移动硬盘）等情况下，会发生变化。

import os
import time
import shutil
import re

# 传入的参数为秒，如果是毫秒，需要/1000
def convertUnixTimeSecondsToDirName(unixTimeSeconds):
    timeFileName = time.strftime('%Y-%m-%d', time.localtime(unixTimeSeconds))
    return timeFileName

# 检测文件名是否包含有日期信息
def detectFileNameTimeString(fileName):
    result = ''
    fileNameParts = re.split(r'[_.-]\s*', fileName)
    # IMG_20200319_213638.jpg or VID_20200315_163030.mp4
    if len(fileNameParts) > 1 and len(fileNameParts[1]) == 8:
        result = fileNameParts[1][0:4] + '-' + fileNameParts[1][4:6] + '-' + fileNameParts[1][6:8]
    # mv_20200315180605.mp4
    elif len(fileNameParts) > 1 and fileNameParts[0] == 'mv' and len(fileNameParts[1]) == 14:
        result = fileNameParts[1][0:4] + '-' + fileNameParts[1][4:6] + '-' + fileNameParts[1][6:8]
    # 2020-03-17-03-28-41-728.jpg
    elif len(fileName) == 27 and len(fileNameParts) == 8:
        result = fileName[0:10]
    #pt2020_03_17_08_02_55.jpg
    #pt2020_03_17_08_02_55_mh1584403415267.jpg
    elif str(fileName).startswith('pt'):
        result = fileName[2:6] + '-' + fileName[7:9] + '-' + fileName[10:12]
    # SL_MO_VID_20200406_204623.mp4
    elif str(fileName).startswith('SL_MO_VID_') and len(fileNameParts) > 3:
        result = fileNameParts[3][0:4] + '-' + fileNameParts[3][4:6] + '-' + fileNameParts[3][6:8]
    # Genie_mh1585312992820.jpg
    elif str(fileName).startswith('Genie_mh') and len(fileName) == 25:
        result = convertUnixTimeSecondsToDirName(int(str('Genie_mh1585312992820.jpg')[8:21]) / 1000)
    # 1585372771448.mp4
    elif len(fileName) == 17 and len(fileNameParts) == 2:
        result = convertUnixTimeSecondsToDirName(int(fileNameParts[0])/1000)
    else:
        print("Not detect:" + fileName)
    return result

def collectFiles():
    SUPPORT_FILE_TYPES = ('.jpg', '.jpeg', '.png', '.mp4', '.mov', '.m4v')
    
    dic = {}
    
    files = [f for f in os.listdir() if f.lower().endswith(SUPPORT_FILE_TYPES)]
    for f in files:
        # 首先检查是否有名字中包含日期信息的，如果有的话，以文件名为准。
        dirName = detectFileNameTimeString(f)
        # 如果文件名中不含有日期信息，则利用文件的时间。
        # 主要考虑到从手机复制文件到计算机中，会导致文件创建时间丢失，所以以文件名为主
        if dirName == '':
            x = os.stat(f)
            dirName = convertUnixTimeSecondsToDirName(x.st_mtime)
            
        if(dirName in dic):
            dic[dirName].append(f)
        else:
            dic[dirName] = []
            dic[dirName].append(f)
            
    for k in dic.keys():
        if not os.path.exists(k):
            os.mkdir(k)
        for f in dic[k]:
            shutil.move(f, k)

if __name__ == '__main__':
    # Set the path to your file path
    # os.chdir('/TestPictures/')
    os.chdir('/Users/gongcen/Pictures/HUAWEI/DCIM/Camera')
    print(os.getcwd() )
    collectFiles()
    print('Finished!')

/Users/gongcen/Pictures/HUAWEI/DCIM/Camera
Finished!
